<a href="https://colab.research.google.com/github/zulmarina1687/MCC-VA-TrabajoFinal/blob/main/NYPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro, to_avro
from pyspark.sql.types import DoubleType
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DecimalType, DoubleType
import pyspark
import pyspark.sql.types as types
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from math import *
import sys
from pyspark.sql.types import LongType, IntegerType, FloatType
from pyspark.sql.functions import *
from IPython.display import display, Math, Latex
from pyspark.sql import functions as F
from datetime import date, timedelta, datetime
import json
from pyspark.sql.types import StructType
from typing import List

spark = SparkSession.builder \
  .master("local[*]") \
  .appName("parquet_example") \
  .getOrCreate()

In [ ]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
   width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}

In [ ]:
#################

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

C:\Users\junior.gomez\AppData\Local\Temp\ipykernel_17028\4038664820.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
############acomodando data dummyy############

In [ ]:
######antes en el colab ya habiamos generado un csv solo con este rango de años a evaluar y esto recien lo leo en pyspark para
#####su transformaciones

In [ ]:
nypd_df = spark.read.csv('NYPD_Complaint_Data_Historic_2018_2023.csv', header = True, sep = ",")


In [ ]:
nypd_df.show(3)

+-----+----------+------------+------------+------------+------------+-----------+----------+-----+-------------+-----+--------------------+----------------+----------+-------+-----------------+-------------+----------------+-----------------+--------+----------+-----------+----------+----------+--------------+--------------+--------+----------------+----------------+-----------------+--------------------+--------------------+------------+-------------+--------------+-------+----+
|ORDEN|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|    OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|        Latitude|        Longitude|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|Year|
+-----+----------+----------

In [ ]:
nypd_df.groupBy('LAW_CAT_CD').count().show()

+-----------+-------+
| LAW_CAT_CD|  count|
+-----------+-------+
|     FELONY| 926314|
|MISDEMEANOR|1477344|
|  VIOLATION| 454910|
+-----------+-------+



In [ ]:
nypd_2.filter(col("CMPLNT_FR_DT").isNull()).count()

In [ ]:
#filtro solo los FELONY.. del

In [ ]:
nypd_felony_df = nypd_df.filter(col("LAW_CAT_CD")=="FELONY")

In [ ]:
nypd_felony_df.count()

926314

In [ ]:
#elimino los campos que no voy a usar

In [ ]:
nypd_2 = nypd_felony_df.drop("ORDEN","CMPLNT_TO_DT","CMPLNT_TO_TM","ADDR_PCT_CD","RPT_DT","KY_CD","PD_CD","JURIS_DESC","JURISDICTION_CODE",
                   "HADEVELOPT","HOUSING_PSA","X_COORD_CD","Y_COORD_CD","TRANSIT_DISTRICT","PATROL_BORO","STATION_NAME","Lat_Lon",
                  "PARKS_NM","CRM_ATPT_CPTD_CD")

In [ ]:
nypd_2.show(5,False)

+----------+------------+------------+----------+---------------+----------+---------+-----------------+-------------+--------------+---------+--------+----------------+-----------------+-------------+--------------+-------+----+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|OFNS_DESC |PD_DESC        |LAW_CAT_CD|BORO_NM  |LOC_OF_OCCUR_DESC|PREM_TYP_DESC|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|Latitude        |Longitude        |VIC_AGE_GROUP|VIC_RACE      |VIC_SEX|Year|
+----------+------------+------------+----------+---------------+----------+---------+-----------------+-------------+--------------+---------+--------+----------------+-----------------+-------------+--------------+-------+----+
|183915130 |06/16/2018  |11:38:00    |ROBBERY   |ROBBERY,BANK   |FELONY    |QUEENS   |INSIDE           |BANK         |45-64         |WHITE    |M       |40.684675       |-73.844567       |UNKNOWN      |UNKNOWN       |D      |2018|
|192830904 |01/18/2019  |17:00:00    |SEX CRIMES|SEXUAL ABUSE   |FELONY    |QUEE

In [ ]:
###############Limpieza datos#####################################

In [ ]:
##########CMPLNT_FR_DT############################

In [ ]:
nypd_2.filter(col("CMPLNT_FR_DT").isNull()).count()

0

In [ ]:
nypd_2.filter(col("CMPLNT_FR_DT")=="(null)").count()

0

In [ ]:
###############CMPLNT_FR_TM#######################

In [ ]:
nypd_2.filter(col("CMPLNT_FR_TM").isNull()).count()

0

In [ ]:
nypd_2.filter(col("CMPLNT_FR_TM")=="(null)").count()

0

In [ ]:
###############OFNS_DESC#######################

In [ ]:
nypd_2.filter(col("OFNS_DESC").isNull()).count()

0

In [ ]:
nypd_2.filter(col("OFNS_DESC")=="(null)").count()

49

In [ ]:
nypd_2_t = nypd_2.withColumn(
    'OFNS_DESC',
    when(col('OFNS_DESC') == '(null)', 'UNKNOWN').otherwise(col('OFNS_DESC'))
)

In [ ]:
nypd_2_t.filter(col("OFNS_DESC")=="(null)").count()

0

In [ ]:
nypd_2_t.groupBy("OFNS_DESC").count().sort(desc("count")).show(5)

+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|       GRAND LARCENY|263458|
|      FELONY ASSAULT|138993|
|             ROBBERY| 87741|
|MISCELLANEOUS PEN...| 84699|
|            BURGLARY| 80601|
+--------------------+------+
only showing top 5 rows



In [ ]:
###############PD_DESC #######################

In [ ]:
nypd_2_t.filter(col("PD_DESC").isNull()).count()

0

In [ ]:
nypd_2_t.filter(col("PD_DESC")=="(null)").count()

2321

In [ ]:
nypd_2_t_ok = nypd_2_t.withColumn(
    'PD_DESC',
    when(col('PD_DESC') == '(null)', 'UNKNOWN').otherwise(col('PD_DESC'))
)

In [ ]:
nypd_2_t_ok.filter(col("PD_DESC")=="(null)").count()

0

In [ ]:
###############LAW_CAT_CD #######################

In [ ]:
nypd_2_t_ok.filter(col("LAW_CAT_CD").isNull()).count()

0

In [ ]:
nypd_2_t_ok.groupBy("LAW_CAT_CD").count().sort(desc("count")).show()

+----------+------+
|LAW_CAT_CD| count|
+----------+------+
|    FELONY|926314|
+----------+------+



In [ ]:
####################BORO_NM  #############################

In [ ]:
nypd_2.filter(col("BORO_NM").isNull()).count()

0

In [ ]:
nypd_2.groupBy("BORO_NM").count().sort(desc("count")).show()

+-------------+------+
|      BORO_NM| count|
+-------------+------+
|     BROOKLYN|272155|
|    MANHATTAN|230527|
|       QUEENS|197713|
|        BRONX|194129|
|STATEN ISLAND| 31693|
|       (null)|    97|
+-------------+------+



In [ ]:
nypd_3 = nypd_2.withColumn(
    'BORO_NM',
    when(col('BORO_NM') == '(null)', 'UNKNOWN').otherwise(col('BORO_NM'))
)

In [ ]:
nypd_3.groupBy("BORO_NM").count().sort(desc("count")).show()

+-------------+------+
|      BORO_NM| count|
+-------------+------+
|     BROOKLYN|272155|
|    MANHATTAN|230527|
|       QUEENS|197713|
|        BRONX|194129|
|STATEN ISLAND| 31693|
|      UNKNOWN|    97|
+-------------+------+



In [ ]:
#############LOC_OF_OCCUR_DESC############################

In [ ]:
nypd_3.filter(col("LOC_OF_OCCUR_DESC").isNull()).count()

0

In [ ]:
nypd_3.groupBy("LOC_OF_OCCUR_DESC").count().sort(desc("count")).show()

+-----------------+------+
|LOC_OF_OCCUR_DESC| count|
+-----------------+------+
|           INSIDE|464369|
|         FRONT OF|247013|
|           (null)|169499|
|      OPPOSITE OF| 24684|
|          REAR OF| 19256|
|          OUTSIDE|  1493|
+-----------------+------+



In [ ]:
nypd_3_t = nypd_3.withColumn(
    'LOC_OF_OCCUR_DESC',
    when(col('LOC_OF_OCCUR_DESC') == '(null)', 'UNKNOWN').otherwise(col('LOC_OF_OCCUR_DESC'))
)

In [ ]:
nypd_3_t.groupBy("LOC_OF_OCCUR_DESC").count().sort(desc("count")).show()

+-----------------+------+
|LOC_OF_OCCUR_DESC| count|
+-----------------+------+
|           INSIDE|464369|
|         FRONT OF|247013|
|          UNKNOWN|169499|
|      OPPOSITE OF| 24684|
|          REAR OF| 19256|
|          OUTSIDE|  1493|
+-----------------+------+



In [ ]:
###############PREM_TYP_DESC#######################

In [ ]:
nypd_3_t.filter(col("PREM_TYP_DESC").isNull()).count()

0

In [ ]:
nypd_3_t.groupBy("PREM_TYP_DESC").count().sort(desc("count")).show(15, False)

+--------------------------+------+
|PREM_TYP_DESC             |count |
+--------------------------+------+
|STREET                    |308824|
|RESIDENCE - APT. HOUSE    |194973|
|RESIDENCE-HOUSE           |87593 |
|RESIDENCE - PUBLIC HOUSING|55713 |
|TRANSIT - NYC SUBWAY      |25513 |
|COMMERCIAL BUILDING       |23416 |
|CHAIN STORE               |20556 |
|OTHER                     |20056 |
|RESTAURANT/DINER          |14812 |
|GROCERY/BODEGA            |14335 |
|DEPARTMENT STORE          |13123 |
|BAR/NIGHT CLUB            |10088 |
|PARK/PLAYGROUND           |9526  |
|CLOTHING/BOUTIQUE         |8189  |
|(null)                    |8104  |
+--------------------------+------+
only showing top 15 rows



In [ ]:
nypd_4 = nypd_3_t.withColumn(
    'PREM_TYP_DESC',
    when(col('PREM_TYP_DESC') == '(null)', 'UNKNOWN').otherwise(col('PREM_TYP_DESC'))
)

In [ ]:
nypd_4.groupBy("PREM_TYP_DESC").count().sort(desc("count")).show(15, False)

+--------------------------+------+
|PREM_TYP_DESC             |count |
+--------------------------+------+
|STREET                    |308824|
|RESIDENCE - APT. HOUSE    |194973|
|RESIDENCE-HOUSE           |87593 |
|RESIDENCE - PUBLIC HOUSING|55713 |
|TRANSIT - NYC SUBWAY      |25513 |
|COMMERCIAL BUILDING       |23416 |
|CHAIN STORE               |20556 |
|OTHER                     |20056 |
|RESTAURANT/DINER          |14812 |
|GROCERY/BODEGA            |14335 |
|DEPARTMENT STORE          |13123 |
|BAR/NIGHT CLUB            |10088 |
|PARK/PLAYGROUND           |9526  |
|CLOTHING/BOUTIQUE         |8189  |
|UNKNOWN                   |8104  |
+--------------------------+------+
only showing top 15 rows



In [ ]:
###############SUSP_AGE_GROUP#######################

In [ ]:
nypd_4.filter(col("SUSP_AGE_GROUP").isNull()).count()

0

In [ ]:
nypd_4.groupBy("SUSP_AGE_GROUP").count().sort(desc("count")).show(10, False)

+--------------+------+
|SUSP_AGE_GROUP|count |
+--------------+------+
|UNKNOWN       |333062|
|25-44         |217636|
|(null)        |215922|
|18-24         |72971 |
|45-64         |64861 |
|<18           |16893 |
|65+           |4898  |
|2022          |8     |
|2018          |6     |
|2021          |6     |
+--------------+------+
only showing top 10 rows



In [ ]:
nypd_5 = nypd_4.withColumn(
    'SUSP_AGE_GROUP',
    when(~col('SUSP_AGE_GROUP').isin("UNKNOWN","25-44","45-64","18-24","<18","65+"), 'UNKNOWN').otherwise(col('SUSP_AGE_GROUP'))
)

In [ ]:
nypd_5.groupBy("SUSP_AGE_GROUP").count().sort(desc("count")).show(10, False)

+--------------+------+
|SUSP_AGE_GROUP|count |
+--------------+------+
|UNKNOWN       |549055|
|25-44         |217636|
|18-24         |72971 |
|45-64         |64861 |
|<18           |16893 |
|65+           |4898  |
+--------------+------+



In [ ]:
###############SUSP_RACE#######################

In [ ]:
nypd_5.filter(col("SUSP_RACE").isNull()).count()

0

In [ ]:
nypd_5.groupBy("SUSP_RACE").count().sort(desc("count")).show(10, False)

+------------------------------+------+
|SUSP_RACE                     |count |
+------------------------------+------+
|BLACK                         |253000|
|UNKNOWN                       |245404|
|(null)                        |215922|
|WHITE HISPANIC                |105639|
|WHITE                         |46716 |
|BLACK HISPANIC                |36418 |
|ASIAN / PACIFIC ISLANDER      |22005 |
|AMERICAN INDIAN/ALASKAN NATIVE|1210  |
+------------------------------+------+



In [ ]:
nypd_6 = nypd_5.withColumn(
    'SUSP_RACE',
    when(col('SUSP_RACE') == '(null)', 'UNKNOWN').otherwise(col('SUSP_RACE'))
)

In [ ]:
nypd_6.groupBy("SUSP_RACE").count().sort(desc("count")).show(10, False)

+------------------------------+------+
|SUSP_RACE                     |count |
+------------------------------+------+
|UNKNOWN                       |461326|
|BLACK                         |253000|
|WHITE HISPANIC                |105639|
|WHITE                         |46716 |
|BLACK HISPANIC                |36418 |
|ASIAN / PACIFIC ISLANDER      |22005 |
|AMERICAN INDIAN/ALASKAN NATIVE|1210  |
+------------------------------+------+



In [ ]:
###############SUSP_SEX#######################|     |

In [ ]:
nypd_6.filter(col("SUSP_SEX").isNull()).count()

0

In [ ]:
nypd_6.groupBy("SUSP_SEX").count().sort(desc("count")).show(10, False)

+--------+------+
|SUSP_SEX|count |
+--------+------+
|M       |420368|
|(null)  |215922|
|U       |215817|
|F       |74207 |
+--------+------+



In [ ]:
nypd_7 = nypd_6.withColumn(
    'SUSP_SEX',
    when(col('SUSP_SEX') == '(null)', 'U').otherwise(col('SUSP_SEX'))
)

In [ ]:
nypd_7.groupBy("SUSP_SEX").count().sort(desc("count")).show(10, False)

+--------+------+
|SUSP_SEX|count |
+--------+------+
|U       |431739|
|M       |420368|
|F       |74207 |
+--------+------+



In [ ]:
###############Latitude#######################

In [ ]:
nypd_7.filter(col("Latitude").isNull()).count()

15

In [ ]:
nypd_7.filter(col("Latitude")=="(null)").count()

0

In [ ]:
###############Longitude#######################

In [ ]:
nypd_7.filter(col("Longitude").isNull()).count()

15

In [ ]:
nypd_7.filter(col("Longitude")=="(null)").count()

0

In [ ]:
###############VIC_AGE_GROUP#######################

In [ ]:
nypd_7.filter(col("VIC_AGE_GROUP").isNull()).count()

0

In [ ]:
nypd_7.groupBy("VIC_AGE_GROUP").count().sort(desc("count")).show(10, False)

+-------------+------+
|VIC_AGE_GROUP|count |
+-------------+------+
|25-44        |333835|
|UNKNOWN      |252099|
|45-64        |169387|
|18-24        |86147 |
|65+          |56261 |
|<18          |28537 |
|-970         |3     |
|-967         |2     |
|-4           |2     |
|-954         |2     |
+-------------+------+
only showing top 10 rows



In [ ]:
nypd_8 = nypd_7.withColumn(
    'VIC_AGE_GROUP',
    when(~col('VIC_AGE_GROUP').isin("UNKNOWN","25-44","45-64","18-24","<18","65+"), 'UNKNOWN').otherwise(col('VIC_AGE_GROUP'))
)

In [ ]:
nypd_8.groupBy("VIC_AGE_GROUP").count().sort(desc("count")).show(10, False)

+-------------+------+
|VIC_AGE_GROUP|count |
+-------------+------+
|25-44        |333835|
|UNKNOWN      |252147|
|45-64        |169387|
|18-24        |86147 |
|65+          |56261 |
|<18          |28537 |
+-------------+------+



In [ ]:
###############VIC_RACE#######################

In [ ]:
nypd_8.filter(col("VIC_RACE").isNull()).count()

0

In [ ]:
nypd_8.groupBy("VIC_RACE").count().sort(desc("count")).show(20, False)

+------------------------------+------+
|VIC_RACE                      |count |
+------------------------------+------+
|UNKNOWN                       |273193|
|BLACK                         |209865|
|WHITE HISPANIC                |161281|
|WHITE                         |154315|
|ASIAN / PACIFIC ISLANDER      |84804 |
|BLACK HISPANIC                |39128 |
|AMERICAN INDIAN/ALASKAN NATIVE|3728  |
+------------------------------+------+



In [ ]:
nypd_9 = nypd_8.withColumn(
    'VIC_RACE',
    when(col('VIC_RACE') == '(null)', 'UNKNOWN').otherwise(col('VIC_RACE'))
)

In [ ]:
nypd_9.groupBy("VIC_RACE").count().sort(desc("count")).show(20, False)

+------------------------------+------+
|VIC_RACE                      |count |
+------------------------------+------+
|UNKNOWN                       |273193|
|BLACK                         |209865|
|WHITE HISPANIC                |161281|
|WHITE                         |154315|
|ASIAN / PACIFIC ISLANDER      |84804 |
|BLACK HISPANIC                |39128 |
|AMERICAN INDIAN/ALASKAN NATIVE|3728  |
+------------------------------+------+



In [ ]:
###############VIC_SEX#######################|      |

In [ ]:
nypd_9.filter(col("VIC_SEX").isNull()).count()

0

In [ ]:
nypd_9.groupBy("VIC_SEX").count().sort(desc("count")).show(20, False)

+-------+------+
|VIC_SEX|count |
+-------+------+
|M      |366031|
|F      |320909|
|D      |128665|
|E      |106458|
|L      |4250  |
|U      |1     |
+-------+------+



In [ ]:
nypd_10 = nypd_9.withColumn(
    'VIC_SEX',
    when(~col('VIC_SEX').isin("F","M","D","E"), 'U').otherwise(col('VIC_SEX'))
)

In [ ]:
nypd_10.groupBy("VIC_SEX").count().sort(desc("count")).show(20, False)

+-------+------+
|VIC_SEX|count |
+-------+------+
|M      |366031|
|F      |320909|
|D      |128665|
|E      |106458|
|U      |4251  |
+-------+------+



In [ ]:
#################

In [ ]:
nypd_10.coalesce(1).write.csv("NYPD_2018_2023_FELONY.csv",header=True)


In [ ]:
######################################################

In [ ]:
nypd_felony_2018_2023 = spark.read.csv('NYPD_2018_2023_FELONY.csv', header = True, sep = ",")

In [ ]:
nypd_felony_2018_2023.show(3)

+----------+------------+------------+----------+------------+----------+---------+-----------------+-------------+--------------+---------+--------+----------------+-----------------+-------------+--------------+-------+----+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM| OFNS_DESC|     PD_DESC|LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|        Latitude|        Longitude|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|Year|
+----------+------------+------------+----------+------------+----------+---------+-----------------+-------------+--------------+---------+--------+----------------+-----------------+-------------+--------------+-------+----+
| 183915130|  06/16/2018|    11:38:00|   ROBBERY|ROBBERY,BANK|    FELONY|   QUEENS|           INSIDE|         BANK|         45-64|    WHITE|       M|       40.684675|       -73.844567|      UNKNOWN|       UNKNOWN|      D|2018|
| 192830904|  01/18/2019|    17:00:00|SEX CRIMES|SEXUAL ABUSE|    FELONY|   QUEENS|         

In [ ]:
nypd_felony_2018_2023_d = nypd_felony_2018_2023.drop('LAW_CAT_CD')

In [ ]:
nypd_felony_2018_2023_d.show(3)

+----------+------------+------------+----------+------------+---------+-----------------+-------------+--------------+---------+--------+----------------+-----------------+-------------+--------------+-------+----+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM| OFNS_DESC|     PD_DESC|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|        Latitude|        Longitude|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|Year|
+----------+------------+------------+----------+------------+---------+-----------------+-------------+--------------+---------+--------+----------------+-----------------+-------------+--------------+-------+----+
| 183915130|  06/16/2018|    11:38:00|   ROBBERY|ROBBERY,BANK|   QUEENS|           INSIDE|         BANK|         45-64|    WHITE|       M|       40.684675|       -73.844567|      UNKNOWN|       UNKNOWN|      D|2018|
| 192830904|  01/18/2019|    17:00:00|SEX CRIMES|SEXUAL ABUSE|   QUEENS|           INSIDE|        OTHER|       UNKNOWN|  UNKNOWN|       

In [ ]:
nypd_felony_2018_2023_d.coalesce(1).write.csv("NYPD_2018_2023_FELONY_D.csv",header=True)

In [ ]:
##########################FINNNNNNNNNNNNNNNNNNNNNNNNNNN#################################

In [ ]:
###############OTROSS........#######################

In [ ]:
nypd_limpio_sin_vic_anio = nypd_limpio_sin_vic.filter(col("Year").isin("2018","2019","2020","2021","2022"))

In [ ]:
nypd_limpio_sin_vic.coalesce(1).write.csv("nypd_limpio_sin_vic.csv",header=True)

In [ ]:
nypd_limpio_sin_vic_anio.coalesce(1).write.csv("nypd_limpio_sin_vic_anio.csv",header=True)

In [ ]:
nypd_limpio_sin_vic_anio_test = nypd_limpio_sin_vic.filter(col("Year").isin("2018","2019"))

In [ ]:
nypd_limpio_sin_vic_anio_test.coalesce(1).write.csv("nypd_limpio_sin_vic_anio_test.csv",header=True)

In [ ]:
###############OTRA LECTURA REDUCIDAD#######################

In [ ]:
nypd_limpio_sin_vic_anio = spark.read.csv('nypd_limpio_sin_vic_anio.csv', header = True, sep = ",")

In [ ]:
nypd_limpio_sin_vic_anio.count()

In [ ]:
###############vvvvvv#######################